In [1]:
# !pip install praatio

In [2]:
import os

### Sample TextGrid files

In [77]:
import tgt

In [80]:
grid_new = tgt.TextGrid()

In [84]:
# grid = tgt.read_textgrid('../samples/libritts/LJ050-0276.TextGrid')

# [i for i in grid.get_tier_by_name('words')]

### Praatio

In [54]:
import praatio
from praatio import textgrid

In [55]:
tg = textgrid.openTextgrid('../samples/libritts/LJ050-0276.TextGrid', False)


In [90]:
tg.tierDict['words'].entryList

[Interval(start=0.0, end=0.18, label='as'),
 Interval(start=0.18, end=0.46, label='has'),
 Interval(start=0.46, end=0.66, label='been'),
 Interval(start=0.66, end=1.07, label='pointed'),
 Interval(start=1.07, end=1.43, label='out'),
 Interval(start=1.79, end=1.9, label='the'),
 Interval(start=1.9, end=2.33, label='commission'),
 Interval(start=2.33, end=2.51, label='has'),
 Interval(start=2.51, end=2.78, label='not'),
 Interval(start=2.78, end=3.35, label='resolved'),
 Interval(start=3.35, end=3.58, label='all'),
 Interval(start=3.58, end=3.69, label='the'),
 Interval(start=3.69, end=4.42, label='proposals'),
 Interval(start=4.42, end=4.65, label='which'),
 Interval(start=4.65, end=4.81, label='could'),
 Interval(start=4.81, end=4.97, label='be'),
 Interval(start=4.97, end=5.45, label='made'),
 Interval(start=6.15, end=6.25, label='the'),
 Interval(start=6.25, end=6.74, label='commission'),
 Interval(start=6.74, end=7.37, label='nevertheless'),
 Interval(start=7.37, end=7.51, label='is

### BuckEye files

In [153]:
with open('../../Buckeye-corpus-part/s13/s1301a.words') as f:
    data = f.read()
    f.close()

In [154]:
with open('../../Buckeye-corpus-part/s13/s1301a.phones') as f:
    data_phones = f.read()
    f.close()

In [155]:
raw_data = [x.strip().split(';')[0].split() for x in data.split('\n')[9:-1]]

In [156]:
raw_data_phones = [x.strip().split(';')[0].split() for x in data_phones.split('\n')[9:-1]]

In [157]:
if raw_data[0][-1]=='{B_TRANS}' and raw_data[-1][-1]=='{E_TRANS}':
    grid_word_tuples = []
    start = 0.0
    for row in raw_data:
        end = row[0]
        tup = (float(start), float(end), row[-1])
        grid_word_tuples.append(tup)
        start = row[0]

    

In [158]:
grid_word_tuples[-1]

(602.829, 602.839075, '{E_TRANS}')

In [159]:
if raw_data_phones[0][-1]=='{B_TRANS}' and raw_data_phones[-1][-1]=='{E_TRANS}':
    grid_phones_tuples = []
    start = 0.0
    for row in raw_data_phones:
        end = row[0]
        tup = (float(start), float(end), row[-1])
        grid_phones_tuples.append(tup)
        start = row[0]

    

#### Create textGrids from this

Ref - https://github.com/timmahrt/praatIO/blob/main/tutorials/tutorial1_intro_to_praatio.ipynb

In [160]:
from praatio import textgrid
tg = textgrid.Textgrid()

In [161]:
# IntervalTiers and PointTiers take four arguments: the tier name, a list of intervals or points,
# a starting time, and an ending time.
wordTier = textgrid.IntervalTier('words', grid_word_tuples, 0, 10.047317916666666)
phoneTier = textgrid.IntervalTier('phones', grid_phones_tuples, 0, 10.047317916666666)


In [162]:
wordTier.entryList[-1]

Interval(start=602.829, end=602.839075, label='{E_TRANS}')

In [163]:
tg.addTier(wordTier)
tg.addTier(phoneTier)

In [164]:
tg.save("empty_textgrid.TextGrid", format="long_textgrid", includeBlankSpaces=False)


In [177]:
grid_word_tuples[-1][1]

602.839075

### Coversion Module

In [179]:
import os
from praatio import textgrid


In [180]:
def build_textgrid(word_file, phones_file):
    
    basename = word_file.split('/')[-1].replace('.words', '')
    
    with open(word_file) as f:
        data = f.read()
        f.close()
        
    with open(phones_file) as f:
        data_phones = f.read()
        f.close()
        
    raw_data = [x.strip().split(';')[0].split() for x in data.split('\n')[9:-1]]
    
    raw_data_phones = [x.strip().split(';')[0].split() for x in data_phones.split('\n')[9:-1]]
    
    grid_word_tuples = []
    if raw_data[0][-1]=='{B_TRANS}' and raw_data[-1][-1]=='{E_TRANS}':
        start = 0.0
        for row in raw_data:
            end = row[0]
            tup = (float(start), float(end), row[-1])
            grid_word_tuples.append(tup)
            start = row[0]
    else:
        print("invalid .words file, aborting formation, grid will be empty")

    grid_phones_tuples = []
    if raw_data_phones[0][-1]=='{B_TRANS}' and raw_data_phones[-1][-1]=='{E_TRANS}':
        start = 0.0
        for row in raw_data_phones:
            end = row[0]
            tup = (float(start), float(end), row[-1])
            grid_phones_tuples.append(tup)
            start = row[0]
    else:
        print("invalid .phones file, aborting formation, grid will be empty")
    
    # Build the grids
    tg = textgrid.Textgrid()
    
    end_time = grid_word_tuples[-1][1]
    wordTier = textgrid.IntervalTier('words', grid_word_tuples, 0, end_time)
    phoneTier = textgrid.IntervalTier('phones', grid_phones_tuples, 0, end_time)

    tg.addTier(wordTier)
    tg.addTier(phoneTier)
    
    tg.save(f"{basename}.TextGrid", format="long_textgrid", includeBlankSpaces=False)


In [181]:
build_textgrid(word_file='../../Buckeye-corpus-part/s13/s1301a.words',
              phones_file='../../Buckeye-corpus-part/s13/s1301a.phones')

### Download utils

In [212]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import requests, zipfile, io

DATA_HOME = "/home/pranav/prominence-estimation-exp/emphases/data/datasets"
DATA_FOLDER = os.path.join(DATA_HOME, "Buckeye")

if not os.path.isdir(DATA_FOLDER):
    os.mkdir(DATA_FOLDER)

def datasets():
    """Download datasets"""
    # TODO - download datasets
    # BuckEye corpus - https://buckeyecorpus.osu.edu/php/speech.php?PHPSESSID=njvutg9l90fc3ebg7v30vnhmk1
    # speakers under consideration
    speakers = {
        's02-1': ["https://buckeyecorpus.osu.edu/speechfiles/s02/s0201a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s02/s0201b.zip"],
        's03-1': ["https://buckeyecorpus.osu.edu/speechfiles/s03/s0301a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s03/s0301b.zip"],
        's04-1': ["https://buckeyecorpus.osu.edu/speechfiles/s04/s0401a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s04/s0401b.zip"],
        's10-1': ["https://buckeyecorpus.osu.edu/speechfiles/s10/s1001a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s10/s1001b.zip"],
        's11-1': ["https://buckeyecorpus.osu.edu/speechfiles/s11/s1101a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s11/s1101b.zip"],
        's14-1': ["https://buckeyecorpus.osu.edu/speechfiles/s14/s1401a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s14/s1401b.zip"],
        's16-1': ["https://buckeyecorpus.osu.edu/speechfiles/s16/s1601a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s16/s1601b.zip"],
        's17-1': ["https://buckeyecorpus.osu.edu/speechfiles/s17/s1701a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s17/s1701b.zip"],
        's21-1': ["https://buckeyecorpus.osu.edu/speechfiles/s21/s2101a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s21/s2101b.zip"],
        's22-1': ["https://buckeyecorpus.osu.edu/speechfiles/s22/s2201a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s22/s2201b.zip"],
        's24-1': ["https://buckeyecorpus.osu.edu/speechfiles/s24/s2401a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s24/s2401b.zip"],
        's25-1': ["https://buckeyecorpus.osu.edu/speechfiles/s25/s2501a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s25/s2501b.zip"],
        's26-1': ["https://buckeyecorpus.osu.edu/speechfiles/s26/s2601a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s26/s2601b.zip"],
        's32-1': ["https://buckeyecorpus.osu.edu/speechfiles/s32/s3201a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s32/s3201b.zip"],
        's33-1': ["https://buckeyecorpus.osu.edu/speechfiles/s33/s3301a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s33/s3301b.zip"],
        's35-1': ["https://buckeyecorpus.osu.edu/speechfiles/s35/s3501a.zip", "https://buckeyecorpus.osu.edu/speechfiles/s35/s3501b.zip"]
    }
    
    for speaker_id in speakers:

        SPEAKER_FOLDER = os.path.join(DATA_FOLDER, speaker_id)
        if not os.path.isdir(SPEAKER_FOLDER):
            os.mkdir(SPEAKER_FOLDER)
            for link in speakers[speaker_id]:
                r = requests.get(link)
                z = zipfile.ZipFile(io.BytesIO(r.content))
                z.extractall(SPEAKER_FOLDER)



In [213]:
datasets()

### Preprocessing utils

In [277]:
import os
import glob
DATA_DIR = "/home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye"

In [280]:
dirc = glob.glob(os.path.join(DATA_DIR, '*/'))
for subdir in dirc:
        words = glob.glob(os.path.join(subdir, '*.words'))
        for word in words:
            basename = word.split('/')[-1].replace('.words', '')
            word_file = os.path.join(subdir, basename+'.words')
            phones_file = os.path.join(subdir, basename+'.phones')
            print(basename, word_file, phones_file, subdir)

s1601b /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s16-1/s1601b.words /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s16-1/s1601b.phones /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s16-1/
s1601a /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s16-1/s1601a.words /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s16-1/s1601a.phones /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s16-1/
s1701b /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s17-1/s1701b.words /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s17-1/s1701b.phones /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s17-1/
s1701a /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s17-1/s1701a.words /home/pranav/prominence-estimation-exp/emphases/data/datasets/Buckeye/s17-1/s1701a.phones /home/pranav/prominence-e